## Library Import

In [1]:
import duckdb
import polars as p

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()

## Load Silver Data

In [2]:
# Test connection to Silver Data db
try:
    db_path = settings['process']['silver_db_path']
    print(f"Attempting to connect to: {db_path}")
    
    # Try connection
    con = duckdb.connect(f"../{db_path}")
    
    # Test Connection with a simple query
    result = con.execute("SELECT 1").fetchone()
    print("Connection Succesfful!")
    print(f"Test query result: {result}")
    
    
    # List all tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("\nAvailable tables:", tables)
    
except Exception as e:
    print(f"Connection Failed {str(e)}")

Attempting to connect to: process/storage/silver/resampled.duckdb
Connection Succesfful!
Test query result: (1,)

Available tables: [('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [3]:
# List all tables
tables = con.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)

Available tables: [('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [4]:
# Get schema information for a specific table
# Replace 'your_table_name' with an actual table name
table_info = con.execute("DESCRIBE silver_1").fetchall()
print("Table schema:", table_info)

Table schema: [('symbol', 'VARCHAR', 'YES', None, None, None), ('date', 'TIMESTAMP WITH TIME ZONE', 'YES', None, None, None), ('open', 'DOUBLE', 'YES', None, None, None), ('high', 'DOUBLE', 'YES', None, None, None), ('low', 'DOUBLE', 'YES', None, None, None), ('close', 'DOUBLE', 'YES', None, None, None), ('volume', 'DOUBLE', 'YES', None, None, None)]


In [5]:
# Get sample data from the table
sample_data = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("Sample data:", sample_data)

Sample data: [('PG', datetime.datetime(2005, 2, 3, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 30.4032161446, 30.4602141802, 30.2607188815, 30.3462181091, 13414900.0), ('PG', datetime.datetime(2005, 4, 4, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 30.3120117508, 30.7794025038, 30.16381341, 30.72240448, 8448400.0), ('PG', datetime.datetime(2005, 5, 17, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 31.5690811662, 31.9300349848, 31.4659515037, 31.8727416992, 7712500.0), ('PG', datetime.datetime(2005, 10, 20, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 32.2396043506, 32.3900934512, 31.8170747977, 31.9328346252, 9112000.0), ('PG', datetime.datetime(2005, 11, 15, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 32.3958788097, 32.4826986697, 32.3669366483, 32.4132423401, 9658100.0)]


In [6]:
# Get the date range of the silver_1 table
con.execute("SELECT MIN(date), MAX(date) FROM silver_1").fetchall() 

[(datetime.datetime(1962, 1, 2, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>),
  datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>))]

In [7]:
# Get the first 5 rows of Resampled Data
first_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)

First 5 rows: [('LPLA', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 388.66, 388.66, 372.715, 380.39, 534557.0), ('GAP', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.12, 22.57, 22.1, 22.25, 11024996.0), ('EVR', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 247.02, 249.13, 244.3125, 247.66, 473503.0), ('ALLT', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 9.0, 9.02, 8.77, 8.95, 261624.0), ('UBXG', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 3.46, 3.595, 3.37, 3.37, 1680.0)]


In [8]:
# Get total row count
row_count = con.execute("SELECT COUNT(*) FROM silver_1").fetchone()
print("Total rows:", row_count)

Total rows: (22513059,)


In [9]:
# Get the first 5 rows of the table
first_5_rows = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)
# Get the last 5 rows of the table
last_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("Last 5 rows:", last_5_rows)


First 5 rows: [('PG', datetime.datetime(2005, 2, 3, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 30.4032161446, 30.4602141802, 30.2607188815, 30.3462181091, 13414900.0), ('PG', datetime.datetime(2005, 4, 4, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 30.3120117508, 30.7794025038, 30.16381341, 30.72240448, 8448400.0), ('PG', datetime.datetime(2005, 5, 17, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 31.5690811662, 31.9300349848, 31.4659515037, 31.8727416992, 7712500.0), ('PG', datetime.datetime(2005, 10, 20, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 32.2396043506, 32.3900934512, 31.8170747977, 31.9328346252, 9112000.0), ('PG', datetime.datetime(2005, 11, 15, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 32.3958788097, 32.4826986697, 32.3669366483, 32.4132423401, 9658100.0)]
Last 5 rows: [('VIAV', datetime.datetime(2025, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/Ed

In [10]:
# Get column statistics
stats = con.execute("""
    SELECT 
        symbol,
        COUNT(*) as count,
        AVG(open) as avg,
        MIN(open) as min,
        MAX(open) as max
    FROM silver_1
    GROUP BY symbol
""").fetchall()
print("Column statistics:", stats)

Column statistics: [('INDB', 9844, 22.70322050464259, 0.0, 85.8686083978), ('PRME', 660, 8.635401356179088, 1.1299999952, 21.2800006866), ('BCPC', 9831, 35.16443097765387, 0.1128147244, 182.6046965802), ('SNEX', 7617, 13.082518410666717, 0.2088889927, 95.01), ('CHR', 1694, 32.57002090540892, 1.0, 106.4000015259), ('ANDE', 7374, 19.72660008246074, 1.4859632818, 59.1059286447), ('OGI', 1522, 6.232063065753154, 0.8799999952, 33.4799995422), ('MHK', 8356, 85.21617917706828, 5.1111111641, 285.049987793), ('IMUX', 2803, 173.64426681703898, 0.68, 1379.1999511719), ('CAAS', 5590, 5.2196598185166, 1.2776288664, 21.4065858471), ('CRNT', 6248, 4.8454051298999214, 0.8799999952, 31.5), ('WMT', 13307, 12.360196035257008, 0.0028309402, 105.0116327221), ('WFRD', 1113, 56.04114301421096, 5.6491760425, 132.230373301), ('ZION', 11401, 21.638407849575405, 0.0, 65.8956851419), ('ORI', 11401, 5.867977049713147, 0.0, 39.5499992371), ('LEGH', 1629, 18.781731735590665, 8.5100002289, 28.7299995422), ('OIS', 611

In [11]:
import polars as pl

# Turn stats into a pl dataframe with mixed types allowed
stats_df = pl.DataFrame(stats, 
                        schema={
                            "symbol": pl.Utf8,
                            "count": pl.Int64,
                            "avg": pl.Float64,
                            "min": pl.Float64,
                            "max": pl.Float64
                        })
print(stats_df)

shape: (5_186, 5)
┌────────┬───────┬────────────┬────────────┬────────────┐
│ symbol ┆ count ┆ avg        ┆ min        ┆ max        │
│ ---    ┆ ---   ┆ ---        ┆ ---        ┆ ---        │
│ str    ┆ i64   ┆ f64        ┆ f64        ┆ f64        │
╞════════╪═══════╪════════════╪════════════╪════════════╡
│ INDB   ┆ 9844  ┆ 22.703221  ┆ 0.0        ┆ 85.868608  │
│ PRME   ┆ 660   ┆ 8.635401   ┆ 1.13       ┆ 21.280001  │
│ BCPC   ┆ 9831  ┆ 35.164431  ┆ 0.112815   ┆ 182.604697 │
│ SNEX   ┆ 7617  ┆ 13.082518  ┆ 0.208889   ┆ 95.01      │
│ CHR    ┆ 1694  ┆ 32.570021  ┆ 1.0        ┆ 106.400002 │
│ …      ┆ …     ┆ …          ┆ …          ┆ …          │
│ FDUS   ┆ 3513  ┆ 8.647913   ┆ 2.654192   ┆ 22.753265  │
│ GEV    ┆ 299   ┆ 273.424553 ┆ 125.310188 ┆ 494.0      │
│ CEPO   ┆ 105   ┆ 10.73405   ┆ 10.04      ┆ 12.7       │
│ UFG    ┆ 101   ┆ 4.473881   ┆ 1.97       ┆ 6.0        │
│ WFF    ┆ 51    ┆ 4.16432    ┆ 3.2        ┆ 5.9        │
└────────┴───────┴────────────┴────────────┴──────────

/opt/homebrew/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


In [12]:
con.close()
